## 남방톱날꽃게(청게)의 나이 예측 딥러닝 모델 구현 2

본 소스 파일에서는 이전의 모델 1과는 다르게 과적합 예방을 위해 K겹 교차 검증 방식(K-fold Cross Validation)을 사용하였다. 과적합을 막기 위해 EarlyStopping() 함수를 이용하여 학습 자동 중단 과정을 설정한 것은 같다.   
다중 선형 회귀 분석의 방법을 사용하여 정확도를 MAE(Mean Absolute Error)를 이용해 평가하였다.

### 1. 필요한 라이브러리 import하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager   # 한글 사용을 위해 import
import numpy as np
import tensorflow as tf

tensorflow 버전 확인하기

In [2]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.3.0


tensorboard import & version 확인하기

In [3]:
import tensorboard

tensorboard.__version__

'2.4.0'

### 2. 데이터셋 가져오기

상대경로를 사용하여 최종적으로 가공된 데이터셋을 가져온다.

In [4]:
filepath = "./3_Dataset.csv"

In [5]:
df = pd.read_csv(filepath)

In [6]:
df

,Length,Diameter,Height,Weight,Viscera Weight,Shell Weight,Age,Sex_F,Sex_I,Sex_M
0,1.4375,1.1750,0.4125,24.635715,5.584852,6.747181,9,1,0,0
1,0.8875,0.6500,0.2125,5.400580,1.374951,1.559222,6,0,0,1
2,1.0375,0.7750,0.2500,7.952035,1.601747,2.764076,6,0,1,0
3,1.1750,0.8875,0.2500,13.480187,2.282135,5.244657,10,1,0,0
4,0.8875,0.6625,0.2125,6.903103,1.488349,1.700970,6,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3847,1.4625,1.1375,0.3250,24.819987,5.854172,6.378637,8,1,0,0
3848,1.5500,1.2125,0.4375,34.458817,7.172423,9.780577,10,1,0,0
3849,0.6250,0.4625,0.1625,2.012815,0.524466,0.637864,5,0,1,0
3850,1.0625,0.7750,0.2625,10.347568,2.338834,2.976698,6,0,1,0


### 3. 딥러닝을 위해 독립변수(x)와 종속변수(y)로 데이터 분리하기

In [7]:
#종속변수 
df_y = df['Age'].values
#독립변수
df_x = df.drop(['Age'], axis=1).values

In [8]:
df_x

array([[1.4375, 1.175 , 0.4125, ..., 1.    , 0.    , 0.    ],
       [0.8875, 0.65  , 0.2125, ..., 0.    , 0.    , 1.    ],
       [1.0375, 0.775 , 0.25  , ..., 0.    , 1.    , 0.    ],
       ...,
       [0.625 , 0.4625, 0.1625, ..., 0.    , 1.    , 0.    ],
       [1.0625, 0.775 , 0.2625, ..., 0.    , 1.    , 0.    ],
       [0.7875, 0.6125, 0.2125, ..., 0.    , 1.    , 0.    ]])

In [9]:
df_y

array([9, 6, 6, ..., 5, 6, 8], dtype=int64)

### 4.  K-겹 교차 검증(K-fold Cross Validation)을 사용한 딥러닝

sklearn에서 제공하는 StratifiedKFold 함수를 사용해 K-겹 교차 검증을 한다.  
StratifiedKFold 함수의 옵션 중에서 n_splits 부분은 K-겹에서 K에 해당하는 값이다.  
본 소스 코드에서는 5로 설정하여 5개의 그룹으로 데이터셋을 나누어 보았다.

In [10]:
from sklearn.model_selection import StratifiedKFold

# seed 값 설정
seed = 2

# k 값 설정
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed) 

In [11]:
skf 

StratifiedKFold(n_splits=5, random_state=2, shuffle=True)

X(독립변수)와 Y(종속변수)의 개수 출력

In [12]:
print("-> X의 개수 : ", len(df_x))
print(df_x)

print("\n-> Y의 개수 : ", len(df_y))
print(df_y)

-> X의 개수 :  3852
[[1.4375 1.175  0.4125 ... 1.     0.     0.    ]
 [0.8875 0.65   0.2125 ... 0.     0.     1.    ]
 [1.0375 0.775  0.25   ... 0.     1.     0.    ]
 ...
 [0.625  0.4625 0.1625 ... 0.     1.     0.    ]
 [1.0625 0.775  0.2625 ... 0.     1.     0.    ]
 [0.7875 0.6125 0.2125 ... 0.     1.     0.    ]]

-> Y의 개수 :  3852
[9 6 6 ... 5 6 8]


학습 데이터 개수와 테스트 데이터 개수 출력 

In [13]:
print("전체 데이터 개수 : ", len(df_x))

# 전체 데이터개수를 k에 해당하는 5로 나누고 반올림
print("각 데이터셋의 개수 : ", round(len(df_x)/5) )

# 학습 데이터의 개수  
print("학습 데이터 개수 : ", round(len(df_x)/5 * 4) )

# 테스트 데이터의 개수  
print("테스트 데이터 개수 : ", (len(df_x)- round(len(df_x)/5 * 4)) )

전체 데이터 개수 :  3852
각 데이터셋의 개수 :  770
학습 데이터 개수 :  3082
테스트 데이터 개수 :  770


### 5. 딥러닝 모델 설계, 컴파일 및 실행하기

MAE를 한 번 시행할 때마다 저장하여 한꺼번에 보여줄 수 있게 mae 리스트를 생성한다.

In [14]:
mae = []
loss = []
stage = 0

아래의 for문은 n_fold만큼 반복된다.  
데이터셋을 skf.split(df_x, df_y)를 이용해 학습 데이터셋과 테스트 데이터셋으로 나눈 후, 각각 train과 test에 할당한다.  
n_fold 마다 셔플이 적용되어 학습 데이터셋과 테스트 데이터셋이 교차될 수 있다.

df_x, df_y는 각각 3852개의 데이터가 있는 상태이며 skf로 셋팅된 환경으로 train, test로 shuffle되어 split된다.

In [15]:
import warnings

warnings.filterwarnings(action='ignore')

학습 조기 종료를 위한 EarlyStopping() 함수

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

In [17]:
for train, test in skf.split(df_x, df_y): # skf로 셋팅된 환경에서 정의한 n_fold만큼 반복
    print("="*14, " training 데이터 개수 : ", len(train), ", test 데이터 개수 : ", len(test), "="*14)
    stage += 1
    print("-> K = %d : %d 번째 그룹에 있는 test 데이터셋 사용 " %(n_fold, stage))
    
    # 딥러닝 구조를 결정(모델을 설정), n_fold 번 반복됨
    # 이전 딥러닝 모델1과 같은 구조로 설정하여 검증 방식에 따른 정확도 차이 비교를 하고자 한다. 
    model = Sequential()
    #1번째 층 : 입력 x는 9개, 출력은 12개, 활성화함수 relu 
    model.add(Dense(12, input_dim=9, activation='relu'))
    #2번째 층 : 입력 x는 12개, 출력은 9개, 활성화함수 relu 
    model.add(Dense(8, activation='relu'))
    #3번째 층 : 입력 x는 9개, 출력은 1개, 활성화함수 linear
    model.add(Dense(1, activation='linear'))
    
    # 모델 컴파일    
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    # 모델 실행
    # 학습 데이터셋을 통한 학습: verbose가 2이면 함축적인 정보만 출력, epochs와 batch_sizes는 이전 모델과 동일하다.
    model.fit(df_x[train], df_y[train], validation_data=(df_x[test],df_y[test]), epochs=50, batch_size=10, verbose=2, callbacks=[early_stopping])
   
    # 테스트 데이터셋을 통한 검증 
    # 테스트 데이터셋 loss, mae(mae로 선형회귀 모델의 정확도 평가)
    k_loss, k_mae = model.evaluate(x=df_x[train], y=df_y[train], verbose=2)    
    # 테스트 데이터셋의 loss
    print("-> Test dataset evaluation : Loss = {:.4f}".format(k_loss))   
    # 테스트 데이터셋의 MAE
    print("-> Test dataset evaluation : MAE = {:.4f}".format(k_mae))
    
    # save k_loss to loss list
    loss.append(k_loss)
    
    # save k_mae to mae list
    mae.append(k_mae)

==============  training 데이터 개수 :  3081 , test 데이터 개수 :  771 ==============
-> K = 5 : 1 번째 그룹에 있는 test 데이터셋 사용 
Epoch 1/50
309/309 - 1s - loss: 26.7387 - mae: 4.1885 - val_loss: 13.7234 - val_mae: 2.9202
Epoch 2/50
309/309 - 0s - loss: 10.4804 - mae: 2.4342 - val_loss: 8.3090 - val_mae: 2.0435
Epoch 3/50
309/309 - 0s - loss: 7.3445 - mae: 1.9618 - val_loss: 6.7035 - val_mae: 1.8381
Epoch 4/50
309/309 - 0s - loss: 6.3312 - mae: 1.8197 - val_loss: 5.9757 - val_mae: 1.7711
Epoch 5/50
309/309 - 0s - loss: 5.7522 - mae: 1.7287 - val_loss: 5.6292 - val_mae: 1.7308
Epoch 6/50
309/309 - 0s - loss: 5.5017 - mae: 1.6807 - val_loss: 5.4148 - val_mae: 1.6231
Epoch 7/50
309/309 - 0s - loss: 5.4396 - mae: 1.6729 - val_loss: 5.5279 - val_mae: 1.7463
Epoch 8/50
309/309 - 0s - loss: 5.3539 - mae: 1.6638 - val_loss: 5.5547 - val_mae: 1.5890
Epoch 9/50
309/309 - 0s - loss: 5.3156 - mae: 1.6479 - val_loss: 5.4142 - val_mae: 1.5881
Epoch 10/50
309/309 - 0s - loss: 5.2968 - mae: 1.6452 - val_loss: 5.2940 -

Epoch 36/50
309/309 - 0s - loss: 5.0673 - mae: 1.5989 - val_loss: 5.2060 - val_mae: 1.5911
Epoch 37/50
309/309 - 0s - loss: 4.9893 - mae: 1.5807 - val_loss: 5.2939 - val_mae: 1.7123
Epoch 38/50
309/309 - 0s - loss: 5.0769 - mae: 1.5955 - val_loss: 5.2477 - val_mae: 1.6947
Epoch 39/50
309/309 - 0s - loss: 5.0751 - mae: 1.6120 - val_loss: 5.1517 - val_mae: 1.6359
Epoch 40/50
309/309 - 0s - loss: 5.0796 - mae: 1.5970 - val_loss: 5.3342 - val_mae: 1.5721
97/97 - 0s - loss: 5.0927 - mae: 1.5258
-> Test dataset evaluation : Loss = 5.0927
-> Test dataset evaluation : MAE = 1.5258
==============  training 데이터 개수 :  3082 , test 데이터 개수 :  770 ==============
-> K = 5 : 3 번째 그룹에 있는 test 데이터셋 사용 
Epoch 1/50
309/309 - 0s - loss: 29.9084 - mae: 4.3002 - val_loss: 15.0925 - val_mae: 3.1210
Epoch 2/50
309/309 - 0s - loss: 14.3638 - mae: 2.9634 - val_loss: 11.8738 - val_mae: 2.6593
Epoch 3/50
309/309 - 0s - loss: 11.3592 - mae: 2.5204 - val_loss: 9.3808 - val_mae: 2.2729
Epoch 4/50
309/309 - 0s - loss: 

Epoch 7/50
309/309 - 0s - loss: 5.8909 - mae: 1.7686 - val_loss: 5.8244 - val_mae: 1.7389
Epoch 8/50
309/309 - 0s - loss: 5.6907 - mae: 1.7330 - val_loss: 5.6762 - val_mae: 1.7857
Epoch 9/50
309/309 - 0s - loss: 5.5109 - mae: 1.6974 - val_loss: 5.5487 - val_mae: 1.7538
Epoch 10/50
309/309 - 0s - loss: 5.4993 - mae: 1.6958 - val_loss: 5.4758 - val_mae: 1.6753
Epoch 11/50
309/309 - 0s - loss: 5.4679 - mae: 1.6875 - val_loss: 5.4935 - val_mae: 1.6489
Epoch 12/50
309/309 - 0s - loss: 5.4301 - mae: 1.6863 - val_loss: 5.4309 - val_mae: 1.7137
Epoch 13/50
309/309 - 0s - loss: 5.4489 - mae: 1.6845 - val_loss: 5.5551 - val_mae: 1.7987
Epoch 14/50
309/309 - 0s - loss: 5.3733 - mae: 1.6698 - val_loss: 5.6036 - val_mae: 1.8177
Epoch 15/50
309/309 - 0s - loss: 5.3141 - mae: 1.6644 - val_loss: 5.4211 - val_mae: 1.6454
Epoch 16/50
309/309 - 0s - loss: 5.3999 - mae: 1.6781 - val_loss: 5.4828 - val_mae: 1.6177
Epoch 17/50
309/309 - 0s - loss: 5.3037 - mae: 1.6458 - val_loss: 5.4488 - val_mae: 1.7721
Ep

### 6. 딥러닝 평가하기

#### Loss(mse)를 K에 해당하는 횟수만큼 출력

In [18]:
print("K 값 :  %.f " % n_fold)
print("Loss : ", loss) #loss 리스트 값 출력

K 값 :  5 
Loss :  [5.108787536621094, 5.092702388763428, 5.218003273010254, 5.110355854034424, 5.631595134735107]


#### MAE를 K에 해당하는 횟수만큼 출력

In [19]:
print("K 값 :  %.f " % n_fold)
print("MAE : ", mae) # mae 리스트 값 출력 

K 값 :  5 
MAE :  [1.6383311748504639, 1.5257914066314697, 1.5927362442016602, 1.5813932418823242, 1.8288325071334839]


#### 테스트 데이터셋을 기반으로 K번 iteration이 적용된 MSE 및 MAE의 평균값 계산

In [20]:
# 리스트형을 넘파이에서 제공하는 배열로 변환 
mse_ave = np.array(loss)
print(type(mse_ave)) 
print("\n MSE : %.4f" %(mse_ave.mean()))  # mse 평균값 출력 

<class 'numpy.ndarray'>

 MSE : 5.2323


In [21]:
mae_ave = np.array(mae)
print(type(mae_ave)) 
print("\n MAE : %.4f" %(mae_ave.mean()))  # mae 평균값 출력 

<class 'numpy.ndarray'>

 MAE : 1.6334


MAE 값이 1.6262가 나왔으므로 꽃게 나이 예측의 평균적인 오차값은 1.6262개월이라는 것을 알 수 있다.

### 7. 새로운 데이터를 위의 딥러닝 모델을 사용하여 예측하기 

keras의 Sequential 모델에서 제공하는 predict() 함수를 사용  
아직 학습과 테스트에 사용하지 않은 데이터를 사용해 예측하였다.

In [22]:
# Length, Diameter, Height, Weight, Viscera Weight, Shell Weight, Sex_F, Sex_I, Sex_M
crab1 = np.array([[1.5305, 1.185, 0.4135, 24.7356155, 5.5858415, 6.749191, 1, 0, 0]])
crab2 = np.array([[0.8975,0.67,0.2305,5.50050975,1.37645075,1.5602225,0,0,1]])

test_crab1 = model.predict(crab1)
test_crab2 = model.predict(crab2)

print("crab1 나이 예측 : %.f" %test_crab1, "개월")
print("crab2 나이 예측 : %.f" %test_crab2, "개월")

crab1 나이 예측 : 11 개월
crab2 나이 예측 : 8 개월


실제 데이터를 바탕으로 만든 가상의 꽃게 데이터에서 딥러닝 모델이 꽤 정확하게 동작함을 확인할 수 있었다.  
crab1은 데이터 셋의 첫 번째 crab 데이터의 각 독립변수들을 조금씩 증가시켜 생후 11개월 정도의 crab을 만들었고, crab2는 데이터 셋의 두 번째 crab 데이터의 각 독립변수들을 조금씩 증가시켜 생후 8개월 정도의 crab을 만든 것이다.

### 8. 학습 모델 저장하기

In [23]:
model.save('CrabAge2.h5') 

### 9. 분석

데이터셋의 꽃게의 나이가 5 ~ 23개월이기 때문에 꽃게 나이 예측의 평균적인 오차값이 1.6262개월인 것은 상당히 예측이 잘 되었다고 생각한다. 그러나 처음 만든 홀드 아웃 교차 검증을 사용한 딥러닝 모델 1(오차값: 1.5869개월)에 비해서 오차값이 크고, K겹 교차 검증 방식 자체가 시간이 상당히 오래 걸리는 검증 방식이기 때문에 비효율적이라고 생각한다. 여기에서 사용한 딥러닝 모델 2의 오차가 더 크게 나타난 이유는 mae 리스트의 4번 째 원소를 보면 알 수 있다. 과적합을 막기 위해 EarlyStopping()을 사용했는데 4번째 iteration에서 10번째로 성능이 개선되지 않는 부분에서 학습을 멈추었고, 그 결과 MAE 값이 1.8284177780151367 라는 상대적으로 큰 수치가 나오게 되었다. 이는 MAE의 평균을 높이는 큰 요인이 되었다. 이를 통해 k겹 교차 검증에서는 EarlyStopping() 함수를 사용하면 우연히 MAE가 큰 지점에서 학습을 멈출 수 있으므로 좋지 않다는 생각이 들었다. 이는 EarlyStopping() 함수를 사용하지 않는 3번 째 모델에서 확인을 할 것이다.